# SUMMARY

In [2]:
from sklearn import preprocessing 
from sklearn.model_selection import GridSearchCV 
from sklearn.ensemble import RandomForestClassifier 
from sklearn.ensemble import RandomForestRegressor

#Import some extention libraries 
%matplotlib inline 
import warnings
import matplotlib.pyplot as plt 
import seaborn as sns 
import numpy as np
import pandas as pd 
import os
import glob
import xlrd
plt.style.use('ggplot')
pd.options.mode.chained_assignment = None

In [3]:
path = r'/Users/dong/Desktop/lwh_rawdata'
dirlist = []
for dirpath,dirname,filename in os.walk(path):
    for i in filename:
        dirlist.append(os.path.join(dirpath,i))

In [4]:
dirlist #你好

['/Users/dong/Desktop/lwh_rawdata/lwh201705.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/lwh201712.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/lwh201704.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/lwh201708.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/042018LWH.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/012018.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/lwh201703.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/lwh20171201.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/lwh201702.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/lwh201701.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/032018LWH.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/lwh201711.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/lwh201707.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/lwh201706.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/lwh201710.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/012018-1.xlsx',
 '/Users/dong/Desktop/lwh_rawdata/022018.xlsx']

In [5]:
dflist = []
for i in dirlist:
    dflist.append(pd.read_excel(i))

In [6]:
mydf = pd.concat(dflist,sort = True)

In [7]:
df = pd.ExcelWriter(r'mydf.xlsx', engine='xlsxwriter',options={'strings_to_urls': False})

In [8]:
mydf.to_excel(df,index=None)

In [9]:
pd.set_option('display.max_columns', None)
#mydf

In [10]:
df = mydf.sort_values("訂單日期", ascending = True) #ascending=False 將序
df.reset_index(inplace=True, drop=True) # rearrange index number 

In [11]:
dPick_up = df
dPick_up['訂單日期'] = pd.to_datetime(dPick_up.訂單日期,format='%Y-%m') # 提取出時間信息
dPick_up['month'] = dPick_up.訂單日期.values.astype('datetime64[M]')

# Filter 

In [12]:
df_filter = df
df_filter = df_filter.loc[df_filter["訂單狀態"] != "已取消"]
#df_filter = df_filter.loc[df_filter["送貨狀態"] != "已退貨"]
#df_filter = df_filter.loc[df_filter["送貨狀態"] != "退貨中"]
#df_filter = df_filter.loc[df_filter["串接物流貨態"] != "未執行"]
#df_filter = df_filter.loc[df_filter["付款狀態"] != "未付款"]

In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 109671 entries, 0 to 109670
Data columns (total 67 columns):
Custom Fields 1 (以下訂單備註，大多都會在出貨的時候才會看見你的留言，請有客服詢問需求多多使用留言或是詢問)    0 non-null float64
Custom Fields 2                                                   0 non-null float64
Custom Fields 3                                                   0 non-null float64
Custom Fields 4                                                   0 non-null float64
Custom Fields 5                                                   0 non-null float64
串接物流貨態                                                            109671 non-null object
交易編號                                                              18300 non-null object
付款方式                                                              109671 non-null object
付款日期                                                              96888 non-null datetime64[ns]
付款狀態                                                              109671 non-null object
付款總金額        

In [14]:
df_filter.付款方式.value_counts()

7-11取貨付款        79756
信用卡（經由綠界)       10125
信用卡付款            6205
ATM櫃員機（經由綠界)     1154
銀行轉帳／ATM          832
宅配貨到付款            650
台新信用卡             165
銀行轉帳(只限宅配使用)       16
特殊付款方式              3
Name: 付款方式, dtype: int64

# Business Health

In [15]:
dB = df_filter
dB['訂單日期'] = pd.to_datetime(dB.訂單日期,format='%Y-%m') # 提取出时间信息
dB['month'] = dB.訂單日期.values.astype('datetime64[M]')
dtest = dB
dtest["month"] = dtest["month"].astype(str)
Revenue = dtest.groupby('month').訂單合計.agg(['sum','count', 'mean'])
Revenue = Revenue.rename(columns = {'sum':'Revenue',"count":"Order","mean":"AOV"})
Revenue["AOV"] = Revenue.AOV.round(decimals = 0)
Revenue["Revenue"] = Revenue.Revenue.astype(str)
Products_sold = dtest.groupby('month').數量.agg(['sum'])
Revenue["AOV"] = Revenue["AOV"].astype(int)
Products_sold = Products_sold.rename(columns = {'sum':'Products sold'})
Business_Health = Revenue.join(Products_sold)
Business_Health['Products per order'] = Business_Health['Products sold']/Business_Health['Order']
Business_Health['Products per order']  = Business_Health['Products per order'] .round(decimals = 1)
Business_Health = Business_Health.T
Business_Health = Business_Health.apply(lambda x : x.astype(str))
Business_Health

month,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01
Revenue,2369983.0,2165212.0,2336493.0,1995509.0,3774759.0,4990144.0,5077375.0,5468335.0,5004080.0,5474288.0,6509226.0,6575176.0,4486592.0,5506572.0,4850942.0
Order,2594,2202,2163,1556,3047,4012,3706,4357,3385,4231,5674,5488,3848,4584,4186
AOV,914,983,1080,1282,1239,1244,1370,1255,1478,1294,1147,1198,1166,1201,1159
Products sold,2989,2765,2944,2256,6203,6960,6994,8549,7434,9441,13138,11675,8086,9534,8915
Products per order,1.2,1.3,1.4,1.4,2.0,1.7,1.9,2.0,2.2,2.2,2.3,2.1,2.1,2.1,2.1


# Customer Profile

In [49]:
Customer_amount = dtest.pivot_table(index='電郵',columns='month',values='訂單合計',aggfunc="count").fillna(0)
columns_month = dtest.month.sort_values().astype('str').unique()
Customer_amount.columns = columns_month
Customer_amount = Customer_amount.apply(lambda x : pd.value_counts(x))

Customer_amount = Customer_amount.iloc[1:]#iloc 切片去掉0
Customer_amount.loc['Customers']= Customer_amount.sum()
#Customer_amount.loc['Customers']= Customer_amount.loc['Customers'].astype(int)
Customer_amount.loc['% of repeat orders'] = Customer_amount.iloc[1:].sum()
Customer_amount.loc['customers bought once']= Customer_amount.iloc[0]
Customer_amount.loc['customers bought more than once'] = Customer_amount.loc['Customers']-Customer_amount.loc['customers bought once']
Customer_amount.loc['% of repeat orders'] = Customer_amount.loc['customers bought more than once']/Customer_amount.loc['Customers']
Customer_amount.loc['% of repeat orders']= Customer_amount.loc['% of repeat orders']*100
Customer_amount.loc['% of repeat orders'] = Customer_amount.loc['% of repeat orders'].round(decimals = 1)
#Customer_amount.loc['% of repeat orders'] = Customer_amount.loc['% of repeat orders']+"%"
#Customer_amount = Customer_amount.iloc[-4:]
#Customer_amount.loc['Customers'] = Customer_amount.loc['Customers'].astype(str)
#Customer_amount.loc['customers bought once'] = Customer_amount.loc['customers bought once'].astype(str)
#Customer_amount.loc['customers bought more than once'] = Customer_amount.loc['customers bought more than once'].astype(str)
#Customer_amount.loc['% of repeat orders'] = Customer_amount.loc['% of repeat orders'].astype(str)
#Customer_amount.loc['% of repeat orders'] = Customer_amount.loc['% of repeat orders']+"%"
Customer_amount = Customer_amount[-4:]
Customer_amount = Customer_amount.apply(lambda x : x.astype(str))
Customer_amount.loc['% of repeat orders'] = Customer_amount.loc['% of repeat orders']+"%"
Customer_amount


,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01
Customers,2538.0,2156.0,2068.0,1517.0,2949.0,3864.0,3561.0,4163.0,3252.0,4017.0,5424.0,5221.0,3727.0,4420.0,4010.0
% of repeat orders,2.0%,2.1%,3.8%,2.3%,3.1%,3.6%,3.5%,4.3%,3.8%,4.9%,4.1%,4.5%,3.0%,3.4%,4.0%
customers bought once,2488.0,2111.0,1990.0,1482.0,2859.0,3724.0,3435.0,3985.0,3129.0,3819.0,5199.0,4988.0,3616.0,4271.0,3851.0
customers bought more than once,50.0,45.0,78.0,35.0,90.0,140.0,126.0,178.0,123.0,198.0,225.0,233.0,111.0,149.0,159.0


In [17]:
Customer_amount_test = dtest.pivot_table(index='電郵',columns='month',values='訂單合計',aggfunc="count").fillna(0)
columns_month = dtest.month.sort_values().astype('str').unique()
Customer_amount_test.columns = columns_month
#Customer_amount.loc['customers']= Customer_amount.count()
#Customer_amount.loc['Total Customer']= Pick_up_amount.apply(lambda x if x == 1 x.sum)
#Customer_amount.applymap(lambda x: 1 if x > 1 else np.NaN)
#Customer_amount.loc['Total Customer'] = Customer_amount.count()#applymap(lambda x: 1 if x > 1 else np.NaN)
#Customer_amount_test = Customer_amount_test.apply(lambda x : pd.value_counts(x))
Customer_amount_test

,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01
電郵,,,,,,,,,,,,,,,
002b2r4054@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
0035967@yahoo.com.tw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
007baroque@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
00iming367859@gmil.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
00ko@yahoo.com.tw,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sunnychang@gmail.com,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
010880@yahoo.com.tw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
0113.invincible@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
01160041@me.mcu.edu.tw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0


# Pick up Analysis (含已取消）

In [18]:
dPick_up = df
dPick_up = dPick_up.loc[dPick_up["付款方式"] == "7-11取貨付款"]
dPick_up = dPick_up.rename(columns = {'送貨狀態':'Pick up Analysis'})
Pick_up_amount = dPick_up.pivot_table(index='Pick up Analysis',columns='month',values='訂單合計',aggfunc="count").fillna(0)
columns_month = dPick_up.month.sort_values().astype('str').unique()
Pick_up_amount.columns = columns_month
Pick_up_amount.loc['Total Pick up count']= Pick_up_amount.sum()
Pick_up_amount.loc['No pick up count'] = Pick_up_amount.loc['已退貨']+ Pick_up_amount.loc['退貨中']
Pick_up_amount.loc['% of no pick up'] = Pick_up_amount.loc['No pick up count']/Pick_up_amount.loc['Total Pick up count']
Pick_up_amount.loc['% of no pick up'] = Pick_up_amount.loc['% of no pick up'].round(decimals = 4)
Pick_up_amount.loc['% of no pick up'] = Pick_up_amount.loc['% of no pick up']*100
Pick_up_amount = Pick_up_amount[-3:]
Pick_up_amount = Pick_up_amount.apply(lambda x : x.astype(str))
Pick_up_amount.loc['% of no pick up'] = Pick_up_amount.loc['% of no pick up']+"%"
Pick_up_amount


,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01
Pick up Analysis,,,,,,,,,,,,,,,
Total Pick up count,2736.0,2340.0,2121.0,1532.0,2997.0,3611.0,3231.0,4023.0,3075.0,3864.0,5314.0,4665.0,3401.0,4061.0,3593.0
No pick up count,254.0,209.0,135.0,106.0,206.0,197.0,187.0,237.0,205.0,225.0,303.0,276.0,207.0,222.0,176.0
% of no pick up,9.28%,8.93%,6.36%,6.92%,6.87%,5.46%,5.79%,5.89%,6.67%,5.82%,5.7%,5.92%,6.09%,5.47%,4.9%


In [19]:
dpick = df
dpick['訂單日期'] = pd.to_datetime(dpick.訂單日期,format='%Y-%m') # 提取出时间信息
dpick['month'] = dpick.訂單日期.values.astype('datetime64[M]')
dpick = dpick.dropna(subset = ["訂單合計"])
dpick['超取取貨時間']=pd.to_datetime(dpick['超取取貨時間']) 
dpick['超取到達時間']=pd.to_datetime(dpick['超取到達時間']) 
#['超取到達時間'] = pd.to_datetime(dy.超取到達時間,format='%Y-%m-%d') # 提取出时间信息
#dy['arrival'] = dy.超取到達時間.values.astype('datetime64[D]')
#dy['超取取貨時間'] = pd.to_datetime(dy.超取取貨時間,format='%Y-%m-%d') # 提取出时间信息
#dy['pickup'] = dy.超取取貨時間.values.astype('datetime64[D]')
dpick['pickupday'] = dpick['超取取貨時間']-dpick['超取到達時間']
dpick = dpick.dropna(subset = ["pickupday"])
dpick.pickupday = dpick.pickupday.astype(str)
split = pd.DataFrame((x.split('d') for x in dpick['pickupday']),index=dpick.index,columns=['day','min'])
dpick = pd.merge(dpick,split,right_index=True, left_index=True)
dpick["month"] = dpick["month"].astype(str)
dpick["day"] = dpick["day"].astype(int)
days = dpick.groupby('month').day.agg(['mean'])
days = days.rename(columns = {'mean':'Avg pick up day'})
Avg_day = days.T
Avg_day = Avg_day.round(decimals = 2)#.apply(lambda x : x.astype(str))
Avg_day = Avg_day.apply(lambda x : x.astype(str))

In [50]:
Pick_up_overview = pd.concat([Pick_up_amount,Avg_day]) 
#Pick_up_overview

In [51]:
pd.concat([Business_Health,Customer_amount,Pick_up_overview]) 

month,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01
Revenue,2369983.0,2165212.0,2336493.0,1995509.0,3774759.0,4990144.0,5077375.0,5468335.0,5004080.0,5474288.0,6509226.0,6575176.0,4486592.0,5506572.0,4850942.0
Order,2594,2202,2163,1556,3047,4012,3706,4357,3385,4231,5674,5488,3848,4584,4186
AOV,914,983,1080,1282,1239,1244,1370,1255,1478,1294,1147,1198,1166,1201,1159
Products sold,2989,2765,2944,2256,6203,6960,6994,8549,7434,9441,13138,11675,8086,9534,8915
Products per order,1.2,1.3,1.4,1.4,2.0,1.7,1.9,2.0,2.2,2.2,2.3,2.1,2.1,2.1,2.1
Customers,2538.0,2156.0,2068.0,1517.0,2949.0,3864.0,3561.0,4163.0,3252.0,4017.0,5424.0,5221.0,3727.0,4420.0,4010.0
% of repeat orders,2.0%,2.1%,3.8%,2.3%,3.1%,3.6%,3.5%,4.3%,3.8%,4.9%,4.1%,4.5%,3.0%,3.4%,4.0%
customers bought once,2488.0,2111.0,1990.0,1482.0,2859.0,3724.0,3435.0,3985.0,3129.0,3819.0,5199.0,4988.0,3616.0,4271.0,3851.0
customers bought more than once,50.0,45.0,78.0,35.0,90.0,140.0,126.0,178.0,123.0,198.0,225.0,233.0,111.0,149.0,159.0
Total Pick up count,2736.0,2340.0,2121.0,1532.0,2997.0,3611.0,3231.0,4023.0,3075.0,3864.0,5314.0,4665.0,3401.0,4061.0,3593.0


# Payment Overview

In [22]:
dtest = dB
dpayment = dtest.dropna(subset = ["訂單合計"])
dpayment = dpayment.rename(columns = {'付款方式':'Payment'})
Payment_amount = dpayment.pivot_table(index='Payment',columns='month',values='訂單合計',aggfunc="count").fillna(0)
columns_month = dpayment.month.sort_values().astype('str').unique()
Payment_amount.columns = columns_month
Payment_amount_percentage = Payment_amount.apply(lambda x : x.astype(int))
#Payment_amount.loc["% of Payment"] = ""
Payment_amount
#Payment_amount_percentage = Payment_amount.apply(lamb.sort_values("2017-01-01", ascending = False)da x : x/x.sum())
#pd.concat([Payment_amount,Payment_amount_percentage])  #上下合併


#pd.merge(Payment_amount,Payment_amount_percentage) 
#Payment_amount = Payment_amount.merge(Payment_amount_percentage)

,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01
Payment,,,,,,,,,,,,,,,
7-11取貨付款,2429.0,2054.0,1919.0,1380.0,2655.0,3319.0,2916.0,3592.0,2745.0,3463.0,4799.0,4496.0,3291.0,3964.0,3518.0
ATM櫃員機（經由綠界),21.0,13.0,23.0,9.0,49.0,117.0,84.0,114.0,90.0,63.0,64.0,41.0,0.0,0.0,0.0
信用卡付款,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,751.0,461.0,527.0,560.0
信用卡（經由綠界),104.0,107.0,157.0,130.0,296.0,502.0,569.0,606.0,550.0,705.0,791.0,0.0,0.0,0.0,0.0
台新信用卡,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20.0,51.0,0.0,0.0,0.0
宅配貨到付款,24.0,28.0,64.0,37.0,47.0,74.0,137.0,45.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
特殊付款方式,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,1.0,0.0,0.0
銀行轉帳(只限宅配使用),16.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
銀行轉帳／ATM,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,147.0,95.0,93.0,108.0


In [23]:
Payment_amount_percentage = Payment_amount.apply(lambda x : x.astype(float))
Payment_amount_percentage = Payment_amount_percentage.apply(lambda x : x/x.sum())
Payment_amount_percentage.apply(lambda x : x.astype(str))
Payment_amount_percentage = Payment_amount_percentage.round(decimals = 2)
Payment_amount_percentage =Payment_amount_percentage*100
Payment_amount_percentage =Payment_amount_percentage.apply(lambda x : x.astype(str)+"%")
Payment_amount_percentage.columns()
Payment_amount_percentage#.sort_values("2017-01-01", ascending = False)

TypeError: 'Index' object is not callable

# Delivery Overview

In [ ]:
dtest = dB
delivery = dtest.dropna(subset = ["訂單合計"])
delivery = delivery.rename(columns = {'送貨方式':'Delivery'})
Delivery_amount = delivery.pivot_table(index='Delivery',columns='month',values='訂單合計',aggfunc="count").fillna(0)
columns_month = delivery.month.sort_values().astype('str').unique()
Delivery_amount.columns = columns_month
#Delivery_amount = Delivery_amount.nlargest(5,"2018-03-01")
Delivery_amount#.sort_values("2017-01-01", ascending = False)

In [ ]:
Delivery_amount.apply(lambda x : x/x.sum())

# Products Overview

In [ ]:
Products_amount = dtest.pivot_table(index='商品名稱',columns='month',values="商品特價",aggfunc="sum").fillna(0)
columns_month = dtest.month.sort_values().astype('str').unique()
Products_amount.columns = columns_month
Products_amount.sort_values("2018-02-01", ascending = True)


In [ ]:
Products_am = Products_amount.T.diff()
Products_am = Products_am.T#.sort_values("2018-02-01", ascending = True)
Products_am[Products_am["2018-02-01"] <  ]

In [ ]:
Products_amounts = Products_amount.T
Products_amounts = Products_amounts.pct_change().T
Products_amounts = Products_amounts.apply(lambda x : x.round(decimals = 2))
Products_amounts = Products_amounts.apply(lambda x : x.astype(float))#+"%")
Products_amounts#.sort_values("2018-02-01", ascending = False)
Products_amounts[Products_amounts["2018-02-01"] == -1 ]

In [ ]:
#Products_amount = dtest.groupby('m').數量.agg(['sum'])

# 消費者各月消費總額

In [33]:
pivoted_amount = dB.pivot_table(index='電郵',columns='month',values='訂單合計',aggfunc='sum').fillna(0)
columns_month = dB.month.sort_values().astype('str').unique()
pivoted_amount.columns = columns_month
pivoted_amount

,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01
電郵,,,,,,,,,,,,,,,
002b2r4054@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,940.0,0.0,0.0,0.0,0.0
0035967@yahoo.com.tw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2694.0,0.0,0.0,0.0
007baroque@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1954.0,0.0,0.0
00iming367859@gmil.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,579.0,0.0
00ko@yahoo.com.tw,0.0,0.0,0.0,0.0,0.0,0.0,2274.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
00sunnychang@gmail.com,0.0,0.0,0.0,0.0,0.0,1330.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
010880@yahoo.com.tw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1119.0
0113.invincible@gmail.com,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,279.0,0.0,0.0,0.0,0.0
01160041@me.mcu.edu.tw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,848.0,0.0,0.0,0.0,0.0,0.0


 # 消費者各月消費  有消費 ＝1 , 無消費=0

In [25]:
pivoted_purchase = pivoted_amount.applymap(lambda x : 1 if x >1 else 0)
pivoted_purchase
#pivoted_purchase_return = pivoted_purchase 
#pivoted_purchase_return

,2017-01-01,2017-02-01,2017-03-01,2017-04-01,2017-05-01,2017-06-01,2017-07-01,2017-08-01,2017-10-01,2017-11-01,2017-12-01,2018-01-01,2018-02-01,2018-03-01,2018-04-01
電郵,,,,,,,,,,,,,,,
002b2r4054@gmail.com,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
0035967@yahoo.com.tw,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0
007baroque@gmail.com,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
00iming367859@gmil.com,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
00ko@yahoo.com.tw,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0
00sunnychang@gmail.com,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0
010880@yahoo.com.tw,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1
0113.invincible@gmail.com,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
01160041@me.mcu.edu.tw,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0


In [52]:
# unreg new active unactive return 

def active_status(user):
    status = []
    for i in range(15):
        if user[i] == 0: # 判斷有無消費 , 等於0無消費  else 不等於0有消費
            if len(status) > 0:  #判斷是否為首月 ,len(status) >0 非首月 else為首月
                if status[i-1] == 'unreg': 
                    status.append('unreg') 
                else:
                    status.append('unactive')
            else: 
                status.append('unreg')
        
        else:
            if len(status) == 0: 
                status.append('new')
            else:
                if status[i-1] == 'unactive':
                    status.append('return')
                elif status[i-1] == 'unreg':
                    status.append('new')
                else:
                    status.append('return')

    return status
pivoted_purchase_status = pivoted_purchase.apply(lambda x: active_status(x),axis = 1)
pivoted_purchase_status.reset_index()

,電郵,0
0,002b2r4054@gmail.com,"[unreg, unreg, unreg, unreg, unreg, unreg, unr..."
1,0035967@yahoo.com.tw,"[unreg, unreg, unreg, unreg, unreg, unreg, unr..."
2,007baroque@gmail.com,"[unreg, unreg, unreg, unreg, unreg, unreg, unr..."
3,00iming367859@gmil.com,"[unreg, unreg, unreg, unreg, unreg, unreg, unr..."
4,00ko@yahoo.com.tw,"[unreg, unreg, unreg, unreg, unreg, unreg, new..."
5,00sunnychang@gmail.com,"[unreg, unreg, unreg, unreg, unreg, new, unact..."
6,010880@yahoo.com.tw,"[unreg, unreg, unreg, unreg, unreg, unreg, unr..."
7,0113.invincible@gmail.com,"[unreg, unreg, unreg, unreg, unreg, unreg, unr..."
8,01160041@me.mcu.edu.tw,"[unreg, unreg, unreg, unreg, unreg, unreg, unr..."
9,0121oliver@gmail.com,"[unreg, unreg, unreg, unreg, unreg, unreg, unr..."


In [47]:
# unreg new active unactive return 

def active_status(user):
    status = []
    for i in range(15):
        if user[i] == 0: # 判斷有無消費 , 等於0無消費  else 不等於0有消費
            if len(status) > 0:  #判斷是否為首月 ,len(status) >0 非首月 else為首月
                if status[i-1] == 'unreg': 
                    status.append('unreg') 
                else:
                    status.append('unactive')
            else: 
                status.append('unreg')
        
        else:
            if len(status) == 0: 
                status.append('new')
            else:
                if status[i-1] == 'unactive':
                    status.append('return')
                elif status[i-1] == 'unreg':
                    status.append('new')
                else:
                    status.append('return')

    return status
pivoted_purchase_status = pivoted_purchase.apply(lambda x: active_status(x),axis = 1, result_type="expand")
pivoted_purchase_status.reset_index()
#pivoted_status = pivoted_counts.apply(active_status,axis=1)
#status_counts =pivoted_status.replace('regis',np.NaN).apply(lambda x:pd.value_counts(x)) #统计计数 regis状态排除掉，它是「未来」才作为新客，这么能计数呢.

,電郵,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,002b2r4054@gmail.com,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,new,unactive,unactive,unactive,unactive
1,0035967@yahoo.com.tw,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,new,unactive,unactive,unactive
2,007baroque@gmail.com,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,new,unactive,unactive
3,00iming367859@gmil.com,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,new,unactive
4,00ko@yahoo.com.tw,unreg,unreg,unreg,unreg,unreg,unreg,new,unactive,unactive,unactive,unactive,unactive,unactive,unactive,unactive
5,00sunnychang@gmail.com,unreg,unreg,unreg,unreg,unreg,new,unactive,unactive,unactive,unactive,unactive,unactive,unactive,unactive,unactive
6,010880@yahoo.com.tw,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,new
7,0113.invincible@gmail.com,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,new,unactive,unactive,unactive,unactive
8,01160041@me.mcu.edu.tw,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,new,unactive,unactive,unactive,unactive,unactive
9,0121oliver@gmail.com,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,unreg,new,unactive,unactive


In [ ]:
#del pivoted_purchase['Col_sum']

In [ ]:
pivoted_purchase['sum'] = pivoted_purchase.iloc[0:].sum(axis=1)
pivoted_purchase = pivoted_purchase['sum']
pivoted_purchase_total = pivoted_purchase.value_counts()
pivoted_purchase_total
pivoted_purchase_summary = pd.DataFrame({'消費次數':pivoted_purchase_total.index,'人數':pivoted_purchase_total.values})
#df = pd.DataFrame({'人數':pivoted_purchase.values},index=list('012345678'))
pivoted_purchase_summary = pivoted_purchase_summary.reindex(columns=['消費次數','人數'])
pivoted_purchase_summary



In [ ]:
pivoted_purchase_percen = pivoted_purchase.value_counts(normalize=True)
pivoted_purchase_percen
dff = pd.DataFrame({'消費次數':pivoted_purchase_percen.index,'人數':pivoted_purchase_percen})
dff.reindex(columns=['消費次數','人數'])


In [48]:
purchase_statue_counts = pivoted_purchase_status.replace("unreg",np.NAN).apply(lambda x : pd.value_counts(x))
purchase_statue_counts

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
new,2538.0,2129,1991,1451,2847,3698,3324,3921,3013,3614,4886,4603,3228,3834,3386
return,NaN,27,77,66,102,166,237,242,239,403,538,618,499,586,624
unactive,NaN,2511,4590,6592,8007,10790,14417,17736,21660,24509,27988,32794,37516,40657,44453


In [ ]:
purchase_statue_counts.fillna(0).T.plot.area(figsize = (12,6))

In [ ]:
return_rata.loc["return"].plot(figsize=(12,6))

In [ ]:
(dB.groupby('電郵').訂單日期.max() - dB.groupby('電郵').訂單日期.min()).describe()

In [ ]:
(dB.groupby('電郵').訂單日期.max()

In [ ]:
((dB.groupby('電郵').訂單日期.max() - dB.groupby('電郵').訂單日期.min())/np.timedelta64(1,'D')).hist(bins=100) 
# 换算的方式直接除timedelta函数即可，这里的np.timedelta64(1, 'D')，D表示天，1表示1天，作为单位使用的

# 消费過兩次以上的老客的生命周期

In [ ]:
life_time =(dB.groupby('電郵').訂單日期.max() - dB.groupby('電郵').訂單日期.min())/np.timedelta64(1,'D')
life_time[life_time > 0].describe()#.hist(bins=200)

In [ ]:
life_time[life_time > 0].hist(bins=200)

In [ ]:
life_timee =(dB.groupby('電郵').訂單日期.max() - dB.groupby('電郵').訂單日期.min()).reset_index()
life_timee.head()



In [ ]:
life_timee["life_time"] = life_timee.訂單日期/np.timedelta64(1,"D")
life_timee[life_timee.life_time > 0 ].life_time.hist(bins = 100 , figsize = (12,6))

In [ ]:
dB.head()

In [ ]:
dB.門市名稱.value_counts().head(10)

In [ ]:
dB.商品名稱.value_counts().head(10)

In [ ]:
dB.商品名稱.value_counts(normalize=True).head(10)

In [ ]:
dB.城市.value_counts().head()

In [ ]:
def topN(dX,n=10):
    counts = dX.value_counts()
    return counts.sort_values(ascending = False)[:n]

In [ ]:
dB.groupby('month').門市名稱.apply(topN).unstack().T

In [ ]:
dB.門市名稱.unique()

In [ ]:
customer_country = dB[['門市名稱','電郵']].drop_duplicates()
Cust_country_count = customer_country.groupby(['門市名稱'])['電郵'].aggregate('count').reset_index().sort_values('電郵', ascending=False)





In [90]:
import matplotlib.pyplot as plt
import seaborn as sns
color = sns.color_palette()


In [ ]:
#Plotting the count of customers
country=list(Cust_country_count['門市名稱'])
Cust_id=list(Cust_country_count['電郵'])
plt.figure(figsize=(12,8))
sns.barplot(country, Cust_id, alpha=0.8, color=color[2])
plt.xticks(rotation='60')
plt.show().head()


# Check unique value for each column.

In [77]:
def unique_counts(df1):
   for i in df1.columns:
       count = df1[i].nunique()
       print(i, ": ", count)
unique_counts(dB)

Custom Fields 1 (以下訂單備註，大多都會在出貨的時候才會看見你的留言，請有客服詢問需求多多使用留言或是詢問) :  0
Custom Fields 2 :  0
Custom Fields 3 :  0
Custom Fields 4 :  0
Custom Fields 5 :  0
串接物流貨態 :  8
交易編號 :  7496
付款方式 :  9
付款日期 :  54977
付款狀態 :  4
付款總金額 :  3869
付款訂單號碼 :  58090
來源訂單號碼 :  0
優惠折扣 :  327
全家服務編號 / 7-11 店號 :  5269
加購品類型 :  1
商品原價 :  562
商品名稱 :  906
商品成本 :  9
商品特價 :  582
商品詳情上次修改時間 :  459
商品貨號 :  281
商品預購提示 :  466
商品類型 :  4
地區/州/省份 :  6
地區／國家 :  10
地址 1 :  2697
城市 :  282
完整地址 :  2700
性別 :  2
折抵購物金 :  21
收件人 :  44846
收件人電話號碼 :  44131
數量 :  13
生日 :  4
自訂折扣合計 :  22
自訂訂單欄位 1 (以下訂單備註，大多都會在出貨的時候才會看見你的留言，請有客服詢問需求多多使用留言或是詢問) :  370
自訂訂單欄位 2 :  0
自訂訂單欄位 3 :  0
自訂訂單欄位 4 :  0
自訂訂單欄位 5 :  0
訂單備註 :  4010
訂單合計 :  3868
訂單小計 :  3763
訂單日期 :  58044
訂單狀態 :  3
訂單號碼 :  58090
貨幣 :  1
超取到達時間 :  392
超取取貨時間 :  52029
超取發貨時間 :  23218
送貨備註 :  23
送貨方式 :  21
送貨時間 :  0
送貨狀態 :  6
送貨編號 :  57965
運費 :  12
選項 :  2062
郵政編號（如適用) :  624
門市名稱 :  4967
附加費 :  3
電話號碼 :  51749
電郵 :  50825
預購商品 :  2
預購訂單 :  2
顧客 :  47322
month :  16


# RFM Customer Segmentation

In [84]:
import datetime as dt
NOW = dt.datetime(2018,5,30)
dB['訂單日期'] = pd.to_datetime(dB['訂單日期'])

In [85]:
rfmTable = dB.groupby('電郵').agg({'訂單日期': lambda x: (NOW - x.max()).days})
rfmTable

,訂單日期
電郵,
002b2r4054@gmail.com,173
0035967@yahoo.com.tw,136
007baroque@gmail.com,94
00iming367859@gmil.com,83
00ko@yahoo.com.tw,320
00sunnychang@gmail.com,339
010880@yahoo.com.tw,30
0113.invincible@gmail.com,170
01160041@me.mcu.edu.tw,188
